<a href="https://colab.research.google.com/github/LivenetsTatiana/works/blob/main/ComputerAlgebra/Exp_Simplification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def add(x, y): return ['+', x, y]
def sub(x, y): return ['-', x, y]
def mul(x, y): return ['*', x, y]
def div(x, y): return ['/', x, y]
def power(x, y): return ['^', x, y]
def negative(x): return ['~', x]
def sin_(x): return ['sin', x]
def cos_(x): return ['cos', x]
def sqrt_(x): return ['sqrt', x]
def ln_(x): return ['ln', x]
def exp_(x): return ['exp', x]
def arcsin_(x): return ['arcsin',x]
def arccos_(x): return ['arccos',x]

def Sum(*x): return ['S', *x]
def Product(*x): return ['P', *x]

In [ ]:
def fix(x):
  if isinstance(x, Expression):
    return x
  return Expression(x)

class Expression:
  def __init__(self, x):
    self.T = x
  def __add__(self, x):
    return Expression(add(self.T, fix(x).T))
  def __radd__(self, x):
    return Expression(add(fix(x).T, self.T))
  def __sub__(self, x):
    return Expression(sub(self.T, fix(x).T))
  def __rsub__(self, x):
    return Expression(sub(fix(x).T, self.T))
  def __mul__(self, x):
    return Expression(mul(self.T, fix(x).T))
  def __rmul__(self, x):
    return Expression(mul(fix(x).T, self.T))
  def __truediv__(self, x):
    return Expression(div(self.T, fix(x).T))
  def __rtruediv__(self, x):
    return Expression(div(fix(x).T, self.T))
  def __pow__(self, x):
    return Expression(power(self.T, fix(x).T))
  def __rpow__(self, x):
    return Expression(power(fix(x).T, self.T))
  def __neg__(self):
    return Expression(negative(self.T))

def sin(x):
  return Expression(sin_(fix(x).T))
def cos(x):
  return Expression(cos_(fix(x).T))
def sqrt(x):
  return Expression(sqrt_(fix(x).T))
def ln(x):
  return Expression(ln_(fix(x).T))
def exp(x):
  return Expression(exp_(fix(x).T))
def arcsin(x):
  return Expression(arcsin_(fix(x).T))
def arccos(x):
  return Expression(arccos_(fix(x).T))

In [ ]:
def show(x):
  display(Math(latex(x)))

def enclose(x):
  return f'\\left( {x} \\right)'

def latex(x):
  if isinstance(x, Expression):
    return latex(x.T)

  if isinstance(x, int):
    return str(x)

  if isinstance(x, str):
    return extendSymbol(x)

  op = x[0]
  if op in '+-*/^':
    l, r = latex(x[1]), latex(x[2])
    if op == '+':
      return f'{l} + {r}'
    if op == '-':
      return f'{l} - {r}'
    if op == '*':
      if isinstance(x[1], list) and x[1][0] in '+-' or l[0] == '-':
        l = enclose(l)
      if isinstance(x[2], list) and x[2][0] in '+-' or r[0] == '-':
        r = enclose(r)
      return f'{l} \\cdot {r}'
    if op == '/':
      return f'\\dfrac{{{l}}}{{{r}}}'

    if op == '^':
      if l == 1 or l == '1': return f'{{{1}}}'
      if isinstance(x[1], list):
        l = enclose(l)

      if r == '0' or r==0 or r == rational(0, 1) or r == ['q', 0, 1]: return f'{{{1}}}'
      elif r == '1' or r == 1: return f'{{{l}}}'
      else: return f'{{{l}}}^{{{r}}}'
  elif op == 'S':
    A = [latex(a) for a in x[1:]]
    for k in range(len(A)):
      if A[k][0] == '-':
        A[k] = enclose(A[k])
      return ' + '.join(A)
  elif op == 'P':
    A = [latex(a) for a in x[1:]]
    for k in range(1, len(x)):
      if isinstance(x[k], list) and x[k][0] == 'S' or A[k-1][0] == '-':
        A[k-1] = enclose(A[k-1])
    return ' \\cdot '.join(A)
  elif op == 'q':
    if x[2] == 1:
      return str(x[1])
    if x[1] > 0:
      return f'\\dfrac{{{x[1]}}}{{{x[2]}}}'
    return f'-\\dfrac{{{-x[1]}}}{{{x[2]}}}'

  else:
    a = enclose(latex(x[1]))
    if op == '~':
      return f'- {a}'
    if op == 'sin':
      return f'\\sin {a}'
    if op == 'cos':
      return f'\\cos {a}'
    if op == 'sqrt':
      return f'\\sqrt{{{a}}}'
    if op == 'ln':
      return f'\\ln {a}'
    if op == 'exp':
      return f'e ^ {{{a}}}'
    if op == 'arcsin':
      return f'\\arcsin {a}'
    if op == 'arccos':
      return f'\\arccos {a}'

def extendSymbol(x):
  if len(x) == 1:
    return x
  if x in ['alpha', 'beta', 'gamma', 'pi']:
    return f'\\{x}'
  return f'{x[0]}_{{{x[1:]}}}'

In [ ]:
def symbols(s):
  S = s.split()
  if len(S) == 1:
    return Expression(s)
  return (Expression(i) for i in S)


In [ ]:
from functools import reduce

In [ ]:
def evalf(x, arg = {}):
  if isinstance(x, Expression):
    if isinstance(arg, dict):
      arg = {i.T: arg[i] for i in arg}
    else:
      arg = {'$': arg}
    return evalf(x.T, arg)

  if isinstance(x, int):
    return x

  if isinstance(x, str):
    if x in arg:
      return arg[x]
    return arg['$']

  op = x[0]
  if op in '+-*/^':
    l, r = evalf(x[1], arg), evalf(x[2], arg)
    if op == '+':
      return l + r
    if op == '-':
      return l - r
    if op == '*':
      return l * r
    if op == '/':
      return l / r
    if op == '^':
      return l ** r
  elif op in 'SP':
    A = [evalf(a, arg) for a in x[1:]]
    if op == 'S':
      return sum(A)
    if op == 'P':
      return reduce(lambda x, y: x * y, A, 1)
  elif op == 'q':
    return x[1] / x[2]
  else:
    a = evalf(x[1], arg)
    if op == '~':
      return - a
    if op == 'sin':
      return np.sin(a)
    if op == 'cos':
      return np.cos(a)
    if op == 'sqrt':
      return np.sqrt(a)
    if op == 'ln':
      return np.log(a)
    if op == 'exp':
      return np.exp(a)
    if op == 'arcsin':
      return np.arcsin(a)
    if op == 'arccos':
      return np.arccos(a)
  return None


In [ ]:
def plot(y, x):
  X = np.linspace(x[1], x[2], 1000)
  if isinstance(y.T, int):
    Y = evalf(y) * np.ones(1000)
  else:
    Y = evalf(y, {x[0]: X})
  plt.plot(X, Y)

In [ ]:
def Z(x):
  return Expression(x)

def isInt(x):
  return isinstance(x.T, int)

def diff(y, x):
  if isInt(y):
    return Z(0)

  if isinstance(y.T, str):
    if y.T == x.T:
      return Z(1)
    return Z(0)

  op = y.T[0]
  if op in '+-*/^':
    u, v = Expression(y.T[1]), Expression(y.T[2])
    du, dv = diff(u, x), diff(v, x)
    if op == '+':
      return du + dv
    if op == '-':
      return du - dv
    if op == '*':
      #if isInt(u): return u * dv # упрощение
      #if isInt(v): return du * v # упрощение
      return du * v + u * dv
    if op == '/':
      if isInt(u): return - u * dv / v ** 2 # упрощение
      if isInt(v): return du / v # упрощение
      return (du * v - u * dv) / v ** 2
    if op == '^':
      if isInt(v): # u(x) ^ n
        n = v.T
        if n == 0:
          return Z(0)
        if n == 1:
          return du
        if du.T == 1: return n * u ** (n - 1) # упрощение
        return n * u ** (n - 1) * du
      if isInt(u): # n ^ v(x)
        n = u.T
        return ln(n) * (u ** v) * dv
  else:
    u = Expression(y.T[1])
    du = diff(u, x)
    if op == '~': return -du
    if op == 'sin': return cos(u) * du
    if op == 'cos': return -sin(u) * du
    if op == 'sqrt': return du / (2 * sqrt(u))
    if op == 'ln': return du / u
    if op == 'exp': return exp(u) * du

  return None

In [ ]:
def taylorCoeffs(y, x, n, x0):
  A = [evalf(y, x0)]
  dy = diff(y, x)
  for i in range(n):
    A.append(evalf(dy, x0))
    dy = diff(dy, x)
  return A

In [ ]:
def fac(n):
  return 1 if n < 2 else n * fac(n - 1)

def taylorTerm(a, x, n, x0):
  a, x0 = int(a), int(x0)
  if n == 0:
    return Z(a)
  d = x
  if x0 > 0:
    d = x - x0
  elif x0 < 0:
    d = x + (-x0)
  if n == 1:
    return a * d
  return (Z(a) / fac(n)) * d ** n

def taylorSeries(y, x, n, x0 = 0):
  A = taylorCoeffs(y, x, n, x0)
  f = None
  for i in range(n + 1):
    if A[i] == 0:
      continue
    elif A[i] > 0:
      t = taylorTerm(A[i], x, i, x0)
      f = t if f == None else f + t
    else:
      t = taylorTerm(-A[i], x, i, x0)
      f = -t if f == None else f - t
  if f == None:
    f = Z(0)
  return f

In [ ]:
def rational(n, m):
  if n == 0:
    return ['q', 0, 1]
  if m < 0:
    n, m = -n, -m
  d = gcd(abs(n), m)
  return ['q', n // d, m // d]

def Q(n, m):
  return Expression(rational(n, m))

def gcd(n, m):
  while m:
    n, m = m, n % m
  return n

In [ ]:
from typing_extensions import Annotated
def normalize(x):
  if isinstance(x, (int, str)): return x
  op, arg = x[0], x[1:]
  A = [normalize(a) for a in arg]
  if op in '+S': # сборка сумм
    R = []
    for a in A:
      R += a[1:] if isinstance(a, list) and a[0] in '+S' else [a]
    return Sum(*R)
  if op in '*P': # сборка произведений
    R = []
    for a in A:
      R += a[1:] if isinstance(a, list) and a[0] in '*P' else [a]
    return Product(*R)
  if op == '~': # унарный минус
    return normalize(Product(-1, A[0]))
  if op == '-': # бинарный минус
    return normalize(Sum(A[0], Product(-1, A[1])))
  if op == '/': # деление
    return normalize(Product(A[0], power(A[1], -1)))
  if op == '^': # нормализация степеней
    if isinstance(A[0], list) and A[0][0] == 'P':
      B = [power(a, A[1]) for a in A[0][1:]]
      return normalize(Product(*B))
    if isinstance(A[0], list) and A[0][0] == '^':
      d = Product(A[0][2], A[1])
      return normalize(power(A[0][1], d))
    if isinstance(A[0], list) and A[0][0] == 'sqrt' and A[1]% 2 ==0:
      return normalize(power(A[0][1], A[1]//2))
  if op == 'sqrt':
    if isinstance(A[0], list) and A[0][0] == '^' and A[0][2] % 2 ==0:
      return normalize(power(A[0][1], A[0][2]//2))

  return x


In [ ]:
def sortx(x):
  if isinstance(x, list):
    op, arg = x[0], x[1:]
    A = [sortx(a) for a in arg]
    if op in 'SP':
      A.sort(key = lambda a: str(a))
    return [op, *A]
  return x

In [ ]:
def simplify(x):
  if isinstance(x, Expression):
    return Expression(simplify(x.T))
  if isinstance(x, str): # символ
    return x
  if isinstance(x, int): # целое, преобразуем в рациональное
    return rational(x, 1)
  if x[0] == 'q': # рациональное
    return x
  x = sortx(normalize(x))
  op, arg = x[0], x[1:]
  A = [simplify(a) for a in arg]

  if op == 'exp':
    if A[0]==['q', 0, 1]: return rational(1, 1)
    if A[0][0]=='ln': return A[0][1]
  if op == 'sin':
    if A[0]==['q', 0, 1]: return rational(0, 1)
    if A[0][0]=='arcsin': return A[0][1]
  if op == 'cos':
    if A[0]==['q', 0, 1]: return rational(1, 1)
    if A[0][0]=='arccos': return A[0][1]
  if op == 'ln':
    if A[0]==['q', 1, 1]: return rational(0, 1)

  if op == '^':
    if isinstance(A[1], list) and A[1][0] == 'q' and A[1][2] == 1:
      if isinstance(A[0], list) and A[0][0] == 'q':
        n, m, d = A[0][1], A[0][2], A[1][1]
        if d >= 0:
          return rational(n ** d, m ** d)
        else:
          return rational(m ** (-d), n ** (-d))
    if A[0] == '1' or A[0] == 1 or A[0] == rational(1, 1) or A[0]==['q', 1, 1]: return rational(1, 1)
    elif A[1] == '1'or A[1]==['q', 1, 1]: return A[0]
    elif A[1] == '0'or A[1]==['q', 0, 1]: return rational(1, 1)
  if op == 'P':
    n, m, V = 1, 1, []
    for a in A:
      if isinstance(a, list) and a[0] == 'q':
        n, m = n * a[1], m * a[2]
      else:
        V.append(a)
    q = rational(n, m)
    if len(V) > 1:
      V0 = collectSameTerms_Product(V)
      V = []
      for i in range(len(V0)):

        if V0[i]=='---':  break
        if V0[i]=='x':V0[i]=['^', 'x', ['q', 1, 1]]
        for j in range(i+1, len(V0)):
          if V0[j]=='x':V0[j]=['^', 'x', ['q', 1, 1]]
          if V0[j][0]==V0[i][0] and V0[j][1]==V0[i][1]:
              ni = V0[i][2][1]
              mi =V0[i][2][2]
              nj = V0[j][2][1]
              mj =V0[j][2][2]
              V0[i][2]=rational(ni*mj+nj*mi, mi*mj)
              V0[j]='---'
        V.append(V0[i])
    # здесь можно упростить множители V, не являющиеся рациональными числами
    if q[1] == 0 or V == []: return q
    if q[1] == 1 and q[2] == 1 and len(V) == 1: return V[0]
    A = [q, *V] if q[1] != 1 or q[2] != 1 else [*V]
  if op == 'S':
    n, m, V = 0, 1, []
    for a in A:
      if isinstance(a, list) and a[0] == 'q':
        n, m = n * a[2] + m * a[1], m * a[2]
      else:
        V.append(a)
    q = rational(n, m)
    if len(V) > 1:
      V0 = collectSameTerms(V)
      V = []
      for i in range(len(V0)):
        if V0[i]=='---':  break
        if V0[i]==[x]:V0[i]=['P', ['q', 1, 1], 'x']
        for j in range(i+1, len(V0)):
          if V0[j]=='x':V0[j]=['P', ['q', 1, 1], 'x']
          if V0[j][0]==V0[i][0] and V0[j][2]==V0[i][2]:
              ni = V0[i][1][1]
              mi =V0[i][1][2]
              nj = V0[j][1][1]
              mj =V0[j][1][2]
              V0[i][1]=rational(ni*mj+nj*mi, mi*mj)
              V0[j]='---'
        V.append(V0[i])
    if V == []: return q
    if q[1] == 0 and len(V) == 1: return V[0]
    A = [q, *V] if q[1] != 0 else [*V]
  return [op, *A]

def collectSameTerms(A):
  count, X = 1, []
  for k in range(1, len(A)):
    if A[k] == A[k - 1]:
      count += 1
      continue
    X.append(simplify(Product(rational(count, 1), A[k - 1])))
    count = 1
  X.append(simplify(Product(rational(count, 1), A[-1])))
  return X
def collectSameTerms_Product(A):
  count, X = 1, []
  for k in range(1, len(A)):
    if A[k] == A[k - 1]:
      count += 1
      continue
    X.append(simplify(Product(rational(count, 1), A[k - 1])))
    count = 1
  X.append(simplify(power(A[-1], rational(count, 1))))
  return X

In [ ]:
x = Expression('x')

2. В функции simplify реализуйте сбор одинаковых множителей в произведении (по ана-
логии со сбором одинаковых слагаемых в суммах):

x · x · x → x
3


In [ ]:
f = x + x+ x
F = simplify(f)
show(F)
F.T

<IPython.core.display.Math object>

['P', ['q', 3, 1], 'x']

In [ ]:
f = x * x* x
F = simplify(f)
show(F)
F.T

<IPython.core.display.Math object>

['^', 'x', ['q', 3, 1]]

3. Добавьте три правила в упрощение степеней: x
0 → 1, 1
x → 1, x
1 → x

In [ ]:
f = x **0
show(simplify(f))
f = x** 1
show(simplify(f))
f = 1**x
show(simplify(f))
f = x**0+x**1+1**x
F = simplify(f)
show(F)
F.T

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

['S', ['q', 2, 1], 'x']

4. Добавьте правила упрощения функций, если их аргументами являются числа со спе-
циальными значениями:

exp(0) → 1, ln(1) → 0, sin(0) → 0

In [ ]:
x0 = Expression(0)
x1 = Expression(1)
f = exp(x0)
show(simplify(f))
f = ln(x1)
show(simplify(f))
f = sin(x0)
show(simplify(f))
f = cos(x0)
show(simplify(f))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

5. Добавьте правила для упрощения взаимно-обратных функций:
exp(ln(x)) → x, sin arcsin(x) → x и т.д.

In [ ]:
f = exp(ln(x))
show(simplify(f))
f = sin(arcsin(x))
show(simplify(f))
f = cos(arccos(x))
show(simplify(f))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

БОНУСЫ


6. Реализуйте более полный вариант приведения общих членов в суммах, с учетом чис-
ловых коэффициентов перед множителями:

x + 5x + 1 − 2x → 1 + 4x.

In [ ]:
f = x*x+2*x
F = simplify(f)
show(F)
F.T

<IPython.core.display.Math object>

['S', ['^', 'x', ['q', 2, 1]], ['P', ['q', 2, 1], 'x']]

7. Реализуйте аналогичный вариант упрощения произведений (однородными теперь будут
множители с одинаковыми основаниями):
x · x
2
· x
5 → x
8

In [ ]:
f = x*x**2*x**5
F = simplify(f)
show(F)
F.T

<IPython.core.display.Math object>

['^', 'x', ['q', 8, 1]]

8. Замените при нормализации функцию извлечения квадратного корня на степень, что-
бы можно было упрощать операции с корнями, используя стандартные приемы для

упрощения степеней:

(
√
x)
4 → x
2
,
√
x
2 → x.

In [ ]:
f = sqrt(x**2)
F = simplify(f)
show(F)

f = sqrt(x)**4
F = simplify(f)
show(F)
F.T

<IPython.core.display.Math object>

<IPython.core.display.Math object>

['^', 'x', ['q', 2, 1]]